reference [https://zhuanlan.zhihu.com/p/32085405]

In [5]:
import torch 
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt 

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01


transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    transform=transform
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size, 
    shuffle=False,
    num_workers=2
)

In [7]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [9]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Test the model
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, sequence_length, input_size).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('* Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), './models/lstm_mnist_model.ckpt')


Epoch [1/2], Step [100/600], Loss: 0.5790
Epoch [1/2], Step [200/600], Loss: 0.3779
Epoch [1/2], Step [300/600], Loss: 0.0982
Epoch [1/2], Step [400/600], Loss: 0.0522
Epoch [1/2], Step [500/600], Loss: 0.1200
Epoch [1/2], Step [600/600], Loss: 0.2050
Test Accuracy of the model on the 10000 test images: 97.14 %
Epoch [2/2], Step [100/600], Loss: 0.1140
Epoch [2/2], Step [200/600], Loss: 0.2017
Epoch [2/2], Step [300/600], Loss: 0.0489
Epoch [2/2], Step [400/600], Loss: 0.0074
Epoch [2/2], Step [500/600], Loss: 0.0806
Epoch [2/2], Step [600/600], Loss: 0.0843
Test Accuracy of the model on the 10000 test images: 97.54 %
